In [ ]:
!pip install numpy rasterio laspy matplotlib

In [ ]:
import laspy
import numpy as np

def load_lidar_points(laz_file: str):
    """
    Loads all LiDAR points (including classification) from a .laz file.

    Parameters
    ----------
    laz_file : str
        Path to the normalized LiDAR .laz file.

    Returns
    -------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Array of classification labels.
    """
    try:
        with laspy.open(laz_file) as las_file:
            las = las_file.read()

        # Convert to NumPy arrays
        x = np.array(las.x)
        y = np.array(las.y)
        z = np.array(las.z)
        classification = np.array(las.classification)  # Keep classification for filtering

        print(f"Loaded {len(x)} LiDAR points from {laz_file}")
        return x, y, z, classification

    except Exception as e:
        raise Exception(f"Error loading LiDAR points: {e}")


In [ ]:
import rasterio
from rasterio.transform import from_origin

def compute_pulse_penetration_ratio(x, y, classification, resolution=10.0):
    """
    Computes the pulse penetration ratio (PPR) in a given grid resolution.

    Parameters
    ----------
    x, y : np.ndarray
        LiDAR point coordinates.
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid resolution in meters (default is 10m).

    Returns
    -------
    ppr_grid : np.ndarray
        The computed pulse penetration ratio grid.
    transform : rasterio.transform.Affine
        Transform for the GeoTIFF output.
    """
    try:
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize grids
        ppr_grid = np.full((len(y_bins)-1, len(x_bins)-1), np.nan)
        total_counts = np.zeros_like(ppr_grid, dtype=np.int32)
        ground_counts = np.zeros_like(ppr_grid, dtype=np.int32)

        # Assign points to grid cells
        x_indices = np.digitize(x, x_bins) - 1
        y_indices = np.digitize(y, y_bins) - 1

        # Count total points and ground points (class 2) per grid cell
        for i in range(len(x)):
            if 0 <= x_indices[i] < ppr_grid.shape[1] and 0 <= y_indices[i] < ppr_grid.shape[0]:
                total_counts[y_indices[i], x_indices[i]] += 1
                if classification[i] == 2:  # Ground point
                    ground_counts[y_indices[i], x_indices[i]] += 1

        # Compute Pulse Penetration Ratio (PPR)
        valid_mask = total_counts > 0  # Avoid division by zero
        ppr_grid[valid_mask] = ground_counts[valid_mask] / total_counts[valid_mask]

        # Flip the grid to align with raster convention
        ppr_grid = np.flipud(ppr_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return ppr_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Pulse Penetration Ratio: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_mean_height(x, y, z, classification, resolution=10.0):
    """
    Computes mean vegetation height in a grid of given resolution.

    Caution: The vegetation mask should be in accordance with your laz data.
    
    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    mean_height_grid : np.ndarray
        The mean vegetation height grid (flipped for correct GeoTIFF output).
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Create empty grid
        mean_height_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Compute mean height per grid cell
        sum_heights = np.zeros_like(mean_height_grid, dtype=np.float64)
        count = np.zeros_like(mean_height_grid, dtype=np.int32)

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < mean_height_grid.shape[1] and 0 <= y_indices[i] < mean_height_grid.shape[0]:
                sum_heights[y_indices[i], x_indices[i]] += z_veg[i]
                count[y_indices[i], x_indices[i]] += 1

        # Compute mean where count > 0
        mean_height_grid[count > 0] = sum_heights[count > 0] / count[count > 0]

        # Flip the Y-axis to align with raster format
        mean_height_grid = np.flipud(mean_height_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return mean_height_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Mean Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_max_height(x, y, z, classification, resolution=10.0):
    """
    Computes the maximum vegetation height in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    max_height_grid : np.ndarray
        The maximum vegetation height grid.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        # vegetation_mask = np.isin(classification, [3, 4, 5])
        # x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Create empty grid
        max_height_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Compute maximum height per grid cell
        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < max_height_grid.shape[1] and 0 <= y_indices[i] < max_height_grid.shape[0]:
                if np.isnan(max_height_grid[y_indices[i], x_indices[i]]):
                    max_height_grid[y_indices[i], x_indices[i]] = z_veg[i]
                else:
                    max_height_grid[y_indices[i], x_indices[i]] = max(max_height_grid[y_indices[i], x_indices[i]], z_veg[i])

        # Flip the Y-axis to align with raster format
        max_height_grid = np.flipud(max_height_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return max_height_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Maximum Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_median_height(x, y, z, classification, resolution=10.0):
    """
    Computes the median vegetation height in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    median_height_grid : np.ndarray
        The median vegetation height grid.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Create empty grid
        median_height_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan, dtype=np.float64)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < median_height_grid.shape[1] and 0 <= y_indices[i] < median_height_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute median height per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            median_height_grid[y_idx, x_idx] = np.median(heights)

        # Flip the Y-axis to align with raster format
        median_height_grid = np.flipud(median_height_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return median_height_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Median Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_height_percentiles(x, y, z, classification, resolution=10.0, percentiles=[25, 50, 75, 95]):
    """
    Computes vegetation height percentiles (25th, 50th, 75th, 95th) in a given grid resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).
    percentiles : list, optional
        List of percentiles to compute (default is [25, 50, 75, 95]).

    Returns
    -------
    percentile_grids : dict
        A dictionary containing percentile grids for each requested percentile.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize grids for each percentile
        percentile_grids = {p: np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan) for p in percentiles}

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < percentile_grids[25].shape[1] and 0 <= y_indices[i] < percentile_grids[25].shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute percentiles per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            for p in percentiles:
                percentile_grids[p][y_idx, x_idx] = np.percentile(heights, p)

        # Flip Y-axis to align with raster format
        for p in percentiles:
            percentile_grids[p] = np.flipud(percentile_grids[p])

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return percentile_grids, transform

    except Exception as e:
        raise Exception(f"Error in computing Vegetation Height Percentiles: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_density_above_mean(x, y, z, classification, resolution=10.0):
    """
    Computes the density of vegetation points above the mean height in a given grid resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    density_above_mean_grid : np.ndarray
        The density of points above the mean height in each grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize grids
        mean_height_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)
        density_above_mean_grid = np.full_like(mean_height_grid, 0, dtype=np.int32)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < mean_height_grid.shape[1] and 0 <= y_indices[i] < mean_height_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute mean height per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            mean_height = np.mean(heights)
            mean_height_grid[y_idx, x_idx] = mean_height
            density_above_mean_grid[y_idx, x_idx] = np.sum(np.array(heights) > mean_height)

        # Flip Y-axis to align with raster format
        density_above_mean_grid = np.flipud(density_above_mean_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return density_above_mean_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Density Above Mean Z: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_br_metrics(x, y, z, classification, resolution=10.0):
    """
    Computes vegetation density ratios (BRs) within height layers.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    br_grids : dict
        A dictionary containing BR grids for each height range.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize grids
        br_keys = ["br_1", "br_1_2", "br_2_3", "br_3_4", "br_4_5", "br_5", "br_5_20", "br_20"]
        br_grids = {key: np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan) for key in br_keys}
        total_counts = np.zeros_like(br_grids["br_1"], dtype=np.int32)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < br_grids["br_1"].shape[1] and 0 <= y_indices[i] < br_grids["br_1"].shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute BRs per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            heights = np.array(heights)
            total_count = len(heights)
            total_counts[y_idx, x_idx] = total_count

            if total_count > 0:
                br_grids["br_1"][y_idx, x_idx] = np.sum((heights >= 0) & (heights < 1)) / total_count
                br_grids["br_1_2"][y_idx, x_idx] = np.sum((heights >= 1) & (heights < 2)) / total_count
                br_grids["br_2_3"][y_idx, x_idx] = np.sum((heights >= 2) & (heights < 3)) / total_count
                br_grids["br_3_4"][y_idx, x_idx] = np.sum((heights >= 3) & (heights < 4)) / total_count
                br_grids["br_4_5"][y_idx, x_idx] = np.sum((heights >= 4) & (heights < 5)) / total_count
                br_grids["br_5"][y_idx, x_idx] = np.sum(heights <= 5) / total_count
                br_grids["br_5_20"][y_idx, x_idx] = np.sum((heights >= 5) & (heights < 20)) / total_count
                br_grids["br_20"][y_idx, x_idx] = np.sum(heights > 20) / total_count

        # Flip Y-axis to align with raster format
        for key in br_grids.keys():
            br_grids[key] = np.flipud(br_grids[key])

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return br_grids, transform

    except Exception as e:
        raise Exception(f"Error in computing BR metrics: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_coeff_var_z(x, y, z, classification, resolution=10.0):
    """
    Computes the coefficient of variation (CV) of vegetation height in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    coeff_var_z_grid : np.ndarray
        The coefficient of variation (CV) of vegetation height per grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]


        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize grid
        coeff_var_z_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < coeff_var_z_grid.shape[1] and 0 <= y_indices[i] < coeff_var_z_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute coefficient of variation per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            heights = np.array(heights)
            mean_z = np.mean(heights)
            std_z = np.std(heights)
            if mean_z > 0:
                coeff_var_z_grid[y_idx, x_idx] = std_z / mean_z
            else:
                coeff_var_z_grid[y_idx, x_idx] = np.nan  # Avoid division by zero

        # Flip Y-axis to align with raster format
        coeff_var_z_grid = np.flipud(coeff_var_z_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return coeff_var_z_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Coefficient of Variation of Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_entropy_z(x, y, z, classification, resolution=10.0, bin_size=0.5):
    """
    Computes the Shannon index (entropy) of vegetation height distribution in a given grid resolution.
    
    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).
    bin_size : float, optional
        Height bin size for computing proportions (default is 0.5m).

    Returns
    -------
    entropy_grid : np.ndarray
        The entropy (Shannon index) of vegetation height distribution per grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize entropy grid
        entropy_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store height distributions per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < entropy_grid.shape[1] and 0 <= y_indices[i] < entropy_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute entropy per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            heights = np.array(heights)
            if len(heights) == 0:
                continue  # Skip empty cells

            # Create height bins (e.g., [0-0.5], [0.5-1.0], etc.)
            max_height = np.max(heights)
            bins = np.arange(0, max_height + bin_size, bin_size)
            hist, _ = np.histogram(heights, bins=bins, density=False)

            # Convert to proportions
            total_points = np.sum(hist)
            proportions = hist / total_points

            # Compute Shannon entropy (ignore bins where p_i = 0)
            entropy = -np.sum(proportions[proportions > 0] * np.log2(proportions[proportions > 0]))

            # Store in grid
            entropy_grid[y_idx, x_idx] = entropy

        # Flip Y-axis to align with raster format
        entropy_grid = np.flipud(entropy_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return entropy_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Shannon Entropy of Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from scipy.stats import kurtosis

def compute_kurtosis_z(x, y, z, classification, resolution=10.0):
    """
    Computes the kurtosis of normalized vegetation height in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    kurtosis_grid : np.ndarray
        The kurtosis of vegetation height per grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize kurtosis grid
        kurtosis_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < kurtosis_grid.shape[1] and 0 <= y_indices[i] < kurtosis_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute kurtosis per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            heights = np.array(heights)
            if len(heights) >= 4:  # Kurtosis requires at least 4 points
                kurtosis_grid[y_idx, x_idx] = kurtosis(heights, fisher=True, nan_policy='omit')
            else:
                kurtosis_grid[y_idx, x_idx] = np.nan  # Not enough data for kurtosis

        # Flip Y-axis to align with raster format
        kurtosis_grid = np.flipud(kurtosis_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return kurtosis_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Kurtosis of Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from sklearn.linear_model import LinearRegression

def compute_sigma_z(x, y, z, classification, resolution=10.0):
    """
    Computes the roughness of vegetation (sigma_z) as the standard deviation 
    of residuals from a locally fitted plane in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    sigma_z_grid : np.ndarray
        The roughness of vegetation per grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize roughness grid
        sigma_z_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store points per grid cell
        grid_points = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < sigma_z_grid.shape[1] and 0 <= y_indices[i] < sigma_z_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_points:
                    grid_points[cell_key] = []
                grid_points[cell_key].append((x_veg[i], y_veg[i], z_veg[i]))

        # Compute roughness per grid cell
        for (y_idx, x_idx), points in grid_points.items():
            points = np.array(points)

            if len(points) >= 3:  # Need at least 3 points to fit a plane
                # Fit a plane using linear regression
                model = LinearRegression()
                XY = points[:, :2]  # Use X and Y as predictors
                Z = points[:, 2]    # Use Z as target variable
                model.fit(XY, Z)
                
                # Compute residuals (actual - predicted height)
                Z_pred = model.predict(XY)
                residuals = Z - Z_pred

                # Compute standard deviation of residuals (roughness)
                sigma_z_grid[y_idx, x_idx] = np.std(residuals)

        # Flip Y-axis to align with raster format
        sigma_z_grid = np.flipud(sigma_z_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return sigma_z_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Sigma Z (Vegetation Roughness): {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from scipy.stats import skew

def compute_h_skew(x, y, z, classification, resolution=10.0):
    """
    Computes the skewness of normalized vegetation height in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    h_skew_grid : np.ndarray
        The skewness of vegetation height per grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize skewness grid
        h_skew_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < h_skew_grid.shape[1] and 0 <= y_indices[i] < h_skew_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute skewness per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            heights = np.array(heights)
            if len(heights) >= 3:  # Skewness requires at least 3 points
                h_skew_grid[y_idx, x_idx] = skew(heights, nan_policy='omit')
            else:
                h_skew_grid[y_idx, x_idx] = np.nan  # Not enough data for skewness

        # Flip Y-axis to align with raster format
        h_skew_grid = np.flipud(h_skew_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return h_skew_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Skewness of Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_h_std(x, y, z, classification, resolution=10.0):
    """
    Computes the standard deviation (Hstd) of normalized vegetation height in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    hstd_grid : np.ndarray
        The standard deviation of vegetation height per grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize standard deviation grid
        hstd_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < hstd_grid.shape[1] and 0 <= y_indices[i] < hstd_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute standard deviation per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            heights = np.array(heights)
            if len(heights) >= 2:  # Standard deviation requires at least 2 points
                hstd_grid[y_idx, x_idx] = np.std(heights, ddof=1)  # ddof=1 for sample std deviation
            else:
                hstd_grid[y_idx, x_idx] = np.nan  # Not enough data

        # Flip Y-axis to align with raster format
        hstd_grid = np.flipud(hstd_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return hstd_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Standard Deviation of Vegetation Height: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

def compute_h_var(x, y, z, classification, resolution=10.0):
    """
    Computes the variance (Hvar) of normalized vegetation height in a grid of given resolution.

    Parameters
    ----------
    x, y, z : np.ndarray
        Arrays of LiDAR point coordinates (X, Y) and normalized height (Z).
    classification : np.ndarray
        Classification labels for each LiDAR point.
    resolution : float, optional
        Grid cell size in meters (default is 10m).

    Returns
    -------
    hvar_grid : np.ndarray
        The variance of vegetation height per grid cell.
    transform : rasterio.transform.Affine
        Transform information for GeoTIFF output.
    """
    try:
        # Extract vegetation points (classification: 3, 4, 5)
        #vegetation_mask = np.isin(classification, [3, 4, 5])
        #x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]

        # For Dutch AHN case, should be used as below:
        vegetation_mask = np.isin(classification, [1])
        x_veg, y_veg, z_veg = x[vegetation_mask], y[vegetation_mask], z[vegetation_mask]
        
        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid bins
        x_bins = np.arange(min_x, max_x + resolution, resolution)
        y_bins = np.arange(min_y, max_y + resolution, resolution)

        # Initialize variance grid
        hvar_grid = np.full((len(y_bins) - 1, len(x_bins) - 1), np.nan)

        # Assign vegetation points to grid cells
        x_indices = np.digitize(x_veg, x_bins) - 1
        y_indices = np.digitize(y_veg, y_bins) - 1

        # Dictionary to store heights per grid cell
        grid_heights = {}

        for i in range(len(x_veg)):
            if 0 <= x_indices[i] < hvar_grid.shape[1] and 0 <= y_indices[i] < hvar_grid.shape[0]:
                cell_key = (y_indices[i], x_indices[i])
                if cell_key not in grid_heights:
                    grid_heights[cell_key] = []
                grid_heights[cell_key].append(z_veg[i])

        # Compute variance per grid cell
        for (y_idx, x_idx), heights in grid_heights.items():
            heights = np.array(heights)
            if len(heights) >= 2:  # Variance requires at least 2 points
                hvar_grid[y_idx, x_idx] = np.var(heights, ddof=1)  # ddof=1 for sample variance
            else:
                hvar_grid[y_idx, x_idx] = np.nan  # Not enough data

        # Flip Y-axis to align with raster format
        hvar_grid = np.flipud(hvar_grid)

        # Define raster transformation
        transform = from_origin(min_x, max_y, resolution, resolution)

        return hvar_grid, transform

    except Exception as e:
        raise Exception(f"Error in computing Variance of Vegetation Height: {e}")


In [ ]:
def save_metric_as_geotiff(metric_grid, transform, output_file, crs="EPSG:28992"):
    """
    Saves a vegetation metric grid as a GeoTIFF file.

    Parameters
    ----------
    metric_grid : np.ndarray
        The computed vegetation metric grid.
    transform : rasterio.transform.Affine
        Transform for geospatial referencing.
    output_file : str
        Output file path for the GeoTIFF.
    crs : str, optional
        Coordinate Reference System (default is EPSG:28992 for RD New).

    Returns
    -------
    None
    """
    try:
        with rasterio.open(
            output_file,
            "w",
            driver="GTiff",
            height=metric_grid.shape[0],
            width=metric_grid.shape[1],
            count=1,
            dtype=metric_grid.dtype,
            crs=crs,
            transform=transform
        ) as dst:
            dst.write(metric_grid, 1)

        print(f"Metric saved: {output_file}")

    except Exception as e:
        raise Exception(f"Error saving GeoTIFF: {e}")


In [ ]:
# Exampl of Usage.


# File path of the normalized LiDAR data
laz_file = "../../Datasets/24HN1_25_normalized.laz"

# Resolution
resolution = 10.0

# Load LiDAR points once
x, y, z, classification = load_lidar_points(laz_file)


# Below are the 25 vegetation metrics to compute and output as geotiff files

'''
# Compute Pulse Penetration Ratio (PPR)
ppr_grid, transform = compute_pulse_penetration_ratio(x, y, classification, resolution)
output_ppr_tif = "../../Datasets/24HN1_25_ppr.tif"
save_metric_as_geotiff(ppr_grid, transform, output_ppr_tif)
'''

'''
# Compute mean vegetation height (10m grid)
mean_height_grid, transform = compute_mean_height(x, y, z, classification, resolution)
output_mean_height_tif = "../../Datasets/24HN1_25_mean_vegetation_height.tif"
save_metric_as_geotiff(mean_height_grid, transform, output_mean_height_tif)
'''

'''
# Compute Maximum Vegetation Height (10m grid)
max_height_grid, transform = compute_max_height(x, y, z, classification, resolution=10.0)
output_max_height_tif = "../../Datasets/24HN1_25_max_vegetation_height.tif"
save_metric_as_geotiff(max_height_grid, transform, output_max_height_tif)
'''

'''
# Compute Median Vegetation Height (10m grid)
median_height_grid, transform = compute_median_height(x, y, z, classification, resolution=10.0)
output_median_height_tif = "../../Datasets/24HN1_25_median_height.tif"
save_metric_as_geotiff(median_height_grid, transform, output_median_height_tif)
'''

'''
# Compute Vegetation Height Percentiles (10m grid)
percentile_grids, transform = compute_height_percentiles(x, y, z, classification, resolution=10.0)
output_percentile_tifs = {
    25: "../../Datasets/24HN1_25_height_25th_percentile.tif",
    50: "../../Datasets/24HN1_25_height_50th_percentile.tif",
    75: "../../Datasets/24HN1_25_height_75th_percentile.tif",
    95: "../../Datasets/24HN1_25_height_95th_percentile.tif"
}

for p, output_file in output_percentile_tifs.items():
    save_metric_as_geotiff(percentile_grids[p], transform, output_file)
'''

'''
# Compute Density Above Mean Z (10m grid)
density_above_mean_grid, transform = compute_density_above_mean(x, y, z, classification, resolution=10.0)
output_density_above_mean_tif="../../Datasets/24HN1_25_density_above_mean_height.tif"
save_metric_as_geotiff(density_above_mean_grid, transform, output_density_above_mean_tif)
'''

'''
# Compute BRs (10m grid)
br_grids, transform = compute_br_metrics(x, y, z, classification, resolution=10.0)
output_br_tifs = {
    "br_1": "../../Datasets/24HN1_25_br_1.tif",
    "br_1_2": "../../Datasets/24HN1_25_br_1_2.tif",
    "br_2_3": "../../Datasets/24HN1_25_br_2_3.tif",
    "br_3_4": "../../Datasets/24HN1_25_br_3_4.tif",
    "br_4_5": "../../Datasets/24HN1_25_br_4_5.tif",
    "br_5": "../../Datasets/24HN1_25_br_5.tif",
    "br_5_20": "../../Datasets/24HN1_25_br_5_20.tif",
    "br_20": "../../Datasets/24HN1_25_br_20.tif",
}
for key, output_file in output_br_tifs.items():
    save_metric_as_geotiff(br_grids[key], transform, output_file)
'''


'''
# Compute Coefficient of Variation of Vegetation Height (10m grid)
coeff_var_z_grid, transform = compute_coeff_var_z(x, y, z, classification, resolution=10.0)
output_coeff_var_z_tif = "../../Datasets/24HN1_25_coefficient_var.tif"
save_metric_as_geotiff(coeff_var_z_grid, transform, output_coeff_var_z_tif)
'''

'''
# Compute Entropy of Vegetation Height (10m grid)
entropy_grid, transform = compute_entropy_z(x, y, z, classification, resolution=10.0, bin_size=0.5)
output_entropy_z_tif="../../Datasets/24HN1_25_entropy_z.tif"
save_metric_as_geotiff(entropy_grid, transform, output_entropy_z_tif)
'''

'''
# Compute Kurtosis of Vegetation Height (10m grid)
kurtosis_grid, transform = compute_kurtosis_z(x, y, z, classification, resolution=10.0)
output_kurtosis_z_tif = "../../Datasets/24HN1_25_kurtosis_z.tif"
save_metric_as_geotiff(kurtosis_grid, transform, output_kurtosis_z_tif)
'''

'''
# Compute Sigma Z (10m grid)
sigma_z_grid, transform = compute_sigma_z(x, y, z, classification, resolution=10.0)
output_sigma_z_tif  = "../../Datasets/24HN1_25_sigma_z.tif"
save_metric_as_geotiff(sigma_z_grid, transform, output_sigma_z_tif)
'''

'''
# Compute Skewness of Vegetation Height (10m grid)
h_skew_grid, transform = compute_h_skew(x, y, z, classification, resolution=10.0)
output_h_skew_tif = "../../Datasets/24HN1_25_h_skew.tif"
save_metric_as_geotiff(h_skew_grid, transform, output_h_skew_tif)
'''

'''
# Compute Standard Deviation of Vegetation Height (10m grid)
hstd_grid, transform = compute_h_std(x, y, z, classification, resolution=10.0)
output_hstd_tif = "../../Datasets/24HN1_25_hstd.tif"
save_metric_as_geotiff(hstd_grid, transform, output_hstd_tif)
'''

'''
# Compute Variance of Vegetation Height (10m grid)
hvar_grid, transform = compute_h_var(x, y, z, classification, resolution=10.0)
output_hvar_tif = "../../Datasets/24HN1_25_h_var.tif"
save_metric_as_geotiff(hvar_grid, transform, output_hvar_tif)
'''
